In [ ]:
%pylab
import numpy as np
import cv2

In [ ]:
image = cv2.imread('foto.jpeg')
cv2.imshow('ImagenOriginal',image)

In [ ]:
#Reescalar la imagen
Nx, Ny = 512,512
image = cv2.resize(image,(Nx,Ny))
#Escala de grises
image_gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
cv2.imshow('ImageGray', image_gray)

image_gray_float = np.float64(image_gray)

In [ ]:
print(image_gray)

In [ ]:
# Obtener FFT
FFT = np.fft.fft2(image_gray_float)
FFT = np.fft.fftshift(FFT) #Centrar las frecuencias en cero

FFTA = np.abs(FFT)

FFTLog = 20*np.log10(FFTA)

print(FFTLog)

In [ ]:
cv2.imshow('EspectroFourier', np.uint8(255*FFTLog/np.max(FFTLog)))

In [ ]:
#Diseñamos nuestro filtro

F1 = np.arange(-0.5*Nx+1, 0.5*Nx+1, 1)
F2 = np.arange(-0.5*Ny+1, 0.5*Ny+1, 1)
[X,Y] = np.meshgrid(F1,F2)

D = np.sqrt(X**2 + Y**2)
D /= np.max(D)
print(D)

In [ ]:
# Definamos un radio de corte
D0 = 0.2
Huv = np.zeros((Nx,Ny))
# Filtro de bajas frecuencias

for i in range(Nx):
    for j in range(Ny):
        if(D[i,j] > D0):
            Huv[i,j] = 1

In [ ]:
cv2.imshow('FiltroDeBajas', np.uint8(255*Huv))

In [ ]:
#Usar el teorema de convolucion
Guv = Huv*FFT
# Tomar la magnitud
GuvA= np.abs(Guv)
#Escalamos
GuvA = np.uint8(255*GuvA/np.max(GuvA))

In [ ]:
cv2.imshow('EspectroConFiltro', np.uint8(255*GuvA))

In [ ]:
# Ahora tomemos la transformada inversa

Gxy = np.fft.ifft2(Guv)
Gxy = np.abs(Gxy)

In [ ]:
cv2.imshow('ImagenConFiltroBajas', np.uint8(Gxy))

# Bordes
Lugares de la imagen donde hay un cambio brusco de la intensidad.

In [ ]:
kernel1 = np.array([-0.5,0.5])
kernel2 = np.array([[-0.5], [0], [0.5]])

In [ ]:
X = cv2.filter2D(image, -1, kernel1)
Y = cv2.filter2D(image, -1, kernel2)

In [ ]:
Mxy = np.sqrt(X**2 + Y**2)
#Normalizacion a 1
Mxy /= np.max(Mxy)

In [ ]:
#Aplicar el filtro y retorna blanco si se cumple la condicion
mask = np.where(Mxy >0.7,255,0)
mask = np.uint8(mask)
cv2.imshow('Bordes', mask)

In [ ]:
video = cv2.VideoCapture(0)

while True:
    
    # Lectura de imagenes
    ret, frame1 = video.read()
    #frame = cv2.cvtColor(frame1, cv2.COLOR_BAYER_BG2RGB)
    
    
    # conversion a flotante
    frame_float = frame1.astype(float)
    
    kernel1 = np.array([-0.5,0,0.5])
    kernel2 = np.array([[-0.5],[0],[0.5]])
    
    X = cv2.filter2D(frame_float, -1, kernel1)
    Y = cv2.filter2D(frame_float, -1, kernel2)
    
    Mxy = np.sqrt(X**2+Y**2)

    # Normalización a 1

    Mxy = Mxy/np.max(Mxy)
    
    mask = np.where(Mxy > 0.1, 255, 0)
    mask = np.uint8(mask)
    
    cv2.imshow('Bordes', mask)
    
    k = cv2.waitKey(1)&0xFF
    if(k == ord('p')):
        print('Exit')
        break
        
video.release()
cv2.destroyAllWindows()